<a href="https://colab.research.google.com/github/dxda6216/Q10_Temp_vs_Activity/blob/main/Q10_temp_vs_activity_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### This is a simple script to calculate Q10 values for activity.
### length by using SciPy Optimize non-linear least squares fit on Colab.
### Copyright (c) 2022 by dxda6216 (dxda6216 AT gmail DOT com)
###
#@title Q10 calculator (Temperature vs. Reaction Rate) {single-column: true}
import numpy as np
import statistics
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt
import pandas as pd


#@markdown This is to caluculate Q10 temperature coefficient for reaction rate, enzymatic activity, frequency, etc (not for circadian period).

#@markdown For the circadian period, please use https://github.com/dxda6216/q10

#@markdown Input data, then hit **Runtime** -> **Run all** (or press **Ctrl+F9**).

# @markdown ---

# Data description (plot title)
Data_description = "Reaction A" #@param {type:"string"}

# Temperature data
Temperatures = 25.0, 30.0, 35.0 #@param {type:"raw"}
x = np.array(Temperatures)

# Period data
Activities = 12.34, 20.68, 28.22  #@param {type:"raw"}
y = np.array(Activities)

xy_data = {'Temperature': x, 'Activity': y}
df = pd.DataFrame(xy_data)

# @markdown ---

# Base temperature
Select_base_temperature = "30°C" #@param ["0\u00B0C", "4\u00B0C", "10\u00B0C", "15\u00B0C", "20\u00B0C", "25\u00B0C", "30\u00B0C", "35\u00B0C", "37\u00B0C", "40\u00B0C", "45\u00B0C", "50\u00B0C", "60\u00B0C", "100\u00B0C", "Minimum", "Maximum", "Average", "Median", "Set \"Base_temperature\" by slider below"]
Base_temperature = 30 # @param {type:"slider", min:0, max:100, step:0.1}

# Mapping of selection to temperature value
temp_mapping = {
    "0\u00B0C": 0.0, "4\u00B0C": 4.0, "10\u00B0C": 10.0, "15\u00B0C": 15.0,
    "20\u00B0C": 20.0, "25\u00B0C": 25.0, "30\u00B0C": 30.0, "35\u00B0C": 35.0,
    "37\u00B0C": 37.0, "40\u00B0C": 40.0, "45\u00B0C": 45.0, "50\u00B0C": 50.0,
    "60\u00B0C": 60.0, "100\u00B0C": 100.0, "Minimum": np.min(x), "Maximum": np.max(x),
    "Average": statistics.mean(x), "Median": statistics.median(x)
}

if Select_base_temperature == "Set \"Base_temperature\" by slider below":
    base_x = Base_temperature
else:
    base_x = temp_mapping[Select_base_temperature]

# Setting the initial fitting parameter Q10 to 1.000
Initial_fitting_parameter_Q10 = 1.000

# Finding the experimental temperature that is the closest to the base
# temperature and the experimental activity at that temperature.
# Setting the initial fitting parameter Activity at Base Temperature to
# that activity value.
closest_x_value_to_bt = min(x, key=lambda z: abs(z - base_x))
df2 = df[df['Temperature'].isin([closest_x_value_to_bt])]
Initial_fitting_parameter_activity_at_base_temperature = df2['Activity'].mean()

p0 = np.array([Initial_fitting_parameter_activity_at_base_temperature, Initial_fitting_parameter_Q10])

# Displaying Tab-delimited data Yes or No
Display_tab_delimited_data = "No" #@param ["No", "Yes"]

# @markdown ---

# Printing the data
print(df, '\n')
print('Base temperatures =', base_x,'\u00B0C\n')

# Defining an equation for curve fitting
# fitting parameters:
#     rate_bt : activity at the base temperatures
#     q10 : temperature coefficient (Q10)
def func(x, rate_bt, q10):
        return ( q10 ** ( ( x - base_x ) * 0.1 ) ) * rate_bt

# Figure graphical_parameter settings
# psx[0] : figure size - width
# psx[1] : figure size - height
# psx[2] : X-axis scale - min
# psx[3] : X-axis scale - max
# psx[4] : X-axis ticks - starting
# psx[5] : X-axis ticks - ending
# psx[6] : X-axis ticks - interval
# psx[7] : Y-axis scale - min
# psx[8] : Y-axis scale - max
# psx[9] : Y-axis ticks - starting
# psx[10] : Y-axis ticks - ending
# psx[11] : Y-axis ticks - interval
# psx[12] : Fitted curve - starting
# psx[13] : Fitted curve - ending
# psx[14] : Marker size
psx = [4, 4, 27, 45, 28, 45, 2, 14, 45, 14, 45, 2, 28, 43, 30]

# Fitting the data to the defined equation
popt, pcov = curve_fit(func, x, y, p0)

residuals = y - func(x, *popt)
ss_residuals = np.sum(residuals**2)
ss_total = np.sum((y-np.mean(y))**2)
r_squared = 1 - ( ss_residuals / ss_total )

# Printing the results
print(u'Estimated activity at', base_x, '\u00B0C =', '{:.3f}'.format(popt[0]), u'\u00B1', '{:.3f}'.format(pcov[0,0]**0.5))
print(u'Q10 (temperature coefficient) =', '{:.3f}'.format(popt[1]), u'\u00B1', '{:.3f}'.format(pcov[1,1]**0.5))
print(u'R\u00B2 =', '{:.6f}'.format(r_squared), u'\n')

# Plotting data and fitted curve
Marker_color = "red"
Fitted_curve_line_color = "lightgreen"
X_axis_caption = "Temperature (\u00B0C)"
Y_axis_caption = "Activity"
fig = plt.figure(figsize = (psx[0], psx[1]))
fcxmin = int( min(x) * 1.25 - max(x) * 0.25 )
fcxmax = int( max(x) * 1.25 - min(x) * 0.25 ) + 1
fcx = np.linspace(fcxmin, fcxmax, 200)

plt.scatter(x, y, s=psx[14], color = Marker_color, label ='data')
fcy = func(fcx, popt[0], popt[1])
plt.plot(fcx, fcy, '--', color=Fitted_curve_line_color, label = 'Fit line    Q10 = %5.3f' %popt[1])
plt.title(Data_description)
plt.xlabel(X_axis_caption)
plt.ylabel(Y_axis_caption)

if Display_tab_delimited_data == "Yes":
        fix = np.array(range(100, 501, 1))
        flx = fix / 10
        fly = func(flx, popt[0], popt[1])
        print(u'Dataset')
        print (u'Temp (\u00B0C)\tActivity')
        ycount = 0
        for xseq in x:
                print(str(xseq)+'\t'+str(y[ycount]))
                ycount += 1
        print(u'\nFitted Curve')
        print (u'Temp (\u00B0C)\tActivity')
        fcycount = 0
        for flxseq in flx:
                print('{:.1f}'.format(flx[fcycount])+'\t'+'{:.3f}'.format(fly[fcycount]))
                fcycount += 1
        print(u'\n')

plt.legend()
plt.show()

### End of script